In [25]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_random_seed(2)

In [4]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [5]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [6]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [7]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [8]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [9]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [10]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split - 1]
y_test = y[split:]

In [11]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)
X_train[:5]

array([[0.60761794, 0.58242373, 0.62172321, 0.51499412, 0.38267307,
        0.46160996, 0.4503991 , 0.5193988 , 0.56445096, 0.55127638],
       [0.58242373, 0.62172321, 0.51499412, 0.38267307, 0.46160996,
        0.4503991 , 0.5193988 , 0.56445096, 0.55127638, 0.5012101 ],
       [0.62172321, 0.51499412, 0.38267307, 0.46160996, 0.4503991 ,
        0.5193988 , 0.56445096, 0.55127638, 0.5012101 , 0.58660203],
       [0.51499412, 0.38267307, 0.46160996, 0.4503991 , 0.5193988 ,
        0.56445096, 0.55127638, 0.5012101 , 0.58660203, 0.5487353 ],
       [0.38267307, 0.46160996, 0.4503991 , 0.5193988 , 0.56445096,
        0.55127638, 0.5012101 , 0.58660203, 0.5487353 , 0.64593307]])

In [12]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.60761794]
  [0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]]

 [[0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]]

 [[0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]]

 [[0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]]

 [[0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]
  [0.64593307]]] 

X_test sample values:
[[[0.04675556]
  [0.03913222]
  [0.03975922]
  [0.03856719]
  [0.03679562]
  [0.03728835]
  [0.03974167]
  [0.04528668]
  [0.04528668]
  [0.07024855]]

 [[0.03913222]
  [0.03975922]
  [0.038567

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

model = Sequential()

number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [15]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [27]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
371/371 [==============================] - 1s 1ms/step - loss: 0.0094
Epoch 2/10
371/371 [==============================] - 0s 694us/step - loss: 0.0094 0s - loss: 0.01
Epoch 3/10
371/371 [==============================] - 0s 825us/step - loss: 0.0094
Epoch 4/10
371/371 [==============================] - 0s 932us/step - loss: 0.0094
Epoch 5/10
371/371 [==============================] - 0s 707us/step - loss: 0.0094
Epoch 6/10
371/371 [==============================] - 0s 746us/step - loss: 0.0094
Epoch 7/10
371/371 [==============================] - 0s 1ms/step - loss: 0.0094A: 0s - loss: 0.01
Epoch 8/10
371/371 [==============================] - 0s 930us/step - loss: 0.0094
Epoch 9/10
371/371 [==============================] - 0s 844us/step - loss: 0.0094
Epoch 10/10
371/371 [==============================] - 0s 934us/step - loss: 0.0094


## Model Performance
In this section, you will evaluate the model using the test data. You will need to:
Evaluate the model using the X_test and y_test data.Use the X_test data to make predictionsCreate a DataFrame of Real (y_test) vs predicted values. Plot the Real vs predicted values as a line chart
Hints
Remember to apply the inverse_transform function to the predicted and y_test values to recover the actual closing prices.

In [18]:
# Evaluate the model
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 1ms/step - loss: 0.0069


0.00687598530203104

In [28]:
# Make some predictions
predicted = model.predict(X_test)

In [ ]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

In [ ]:
# Plot the real vs predicted values as a line chart
stocks.hvplot(title="Real vs. Predicted Stock Price", xlabel="Days", ylabel="Price")